In [1]:
from theano.sandbox import cuda
cuda.use('gpu2')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(2), GPU number 0 is already in use.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


# Setup

In [3]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = np.expand_dims(x_train, 1)
x_test = np.expand_dims(x_test, 1)

In [5]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [6]:
mean_x = x_train.mean().astype(np.float32)
std_x = x_train.std().astype(np.float32)

In [7]:
# method to normalise the input 
def normalise(x): return (x-mean_x)/std_x

# Linear Model

In [10]:
x_train.shape

(60000, 1, 28, 28)

In [11]:
def get_lin_model():
    model = Sequential([
             Lambda(normalise, input_shape=x_train.shape[1:]),
             Flatten(),
             Dense(10, activation='softmax')
    ])
    model.compile(Adam(), 'categorical_crossentropy', ['accuracy'])
    return model

In [12]:
lm = get_lin_model()

In [13]:
#Generate batches from image data
gen = image.ImageDataGenerator()
batches = gen.flow(x_train, y_train, batch_size=64)
val_batches = gen.flow(x_test, y_test, batch_size=64)

In [14]:
#Run the model for 1 epoch using the default learning rate
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4285 - acc: 0.8737 - val_loss: 0.3020 - val_acc: 0.9136


In [15]:
lm.optimizer.lr = 0.1

In [16]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.2988 - acc: 0.9146 - val_loss: 0.2997 - val_acc: 0.9168


In [17]:
lm.optimizer.lr = 0.01

In [18]:
lm.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.2847 - acc: 0.9206 - val_loss: 0.2845 - val_acc: 0.9216
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.2772 - acc: 0.9229 - val_loss: 0.2800 - val_acc: 0.9202
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.2724 - acc: 0.9244 - val_loss: 0.2820 - val_acc: 0.9214
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.2683 - acc: 0.9255 - val_loss: 0.2696 - val_acc: 0.9251


# Single Dense Layer

In [32]:
def get_fc_model():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Flatten(),
            Dense(512, activation='softmax'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
fc = get_fc_model()

In [34]:
fc.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 1.5455 - acc: 0.8841 - val_loss: 1.0126 - val_acc: 0.9206


In [35]:
fc.optimizer.lr=0.1

In [36]:
fc.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 0.7477 - acc: 0.9240 - val_loss: 0.5532 - val_acc: 0.9278


In [37]:
fc.optimizer.lr=0.01

In [38]:
fc.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.4626 - acc: 0.9297 - val_loss: 0.3962 - val_acc: 0.9319
Epoch 2/4
60000/60000 [==============================] - 13s - loss: 0.3513 - acc: 0.9355 - val_loss: 0.3270 - val_acc: 0.9359
Epoch 3/4
60000/60000 [==============================] - 13s - loss: 0.3021 - acc: 0.9382 - val_loss: 0.3106 - val_acc: 0.9319
Epoch 4/4
60000/60000 [==============================] - 13s - loss: 0.2740 - acc: 0.9409 - val_loss: 0.2832 - val_acc: 0.9367


# Basic VGG-style CNN

In [8]:
def get_model():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Convolution2D(32, 3, 3, activation='relu'),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            Convolution2D(64, 3, 3, activation='relu'),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [45]:
model = get_model()

In [46]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1085 - acc: 0.9667 - val_loss: 0.0524 - val_acc: 0.9828


In [47]:
model.optimizer.lr=0.1

In [48]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.0374 - acc: 0.9883 - val_loss: 0.0223 - val_acc: 0.9932


In [49]:
model.optimizer.lr=0.01

In [50]:
model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/8
60000/60000 [==============================] - 20s - loss: 0.0238 - acc: 0.9922 - val_loss: 0.0262 - val_acc: 0.9918
Epoch 2/8
60000/60000 [==============================] - 20s - loss: 0.0188 - acc: 0.9941 - val_loss: 0.0364 - val_acc: 0.9907
Epoch 3/8
60000/60000 [==============================] - 20s - loss: 0.0149 - acc: 0.9953 - val_loss: 0.0265 - val_acc: 0.9931
Epoch 4/8
60000/60000 [==============================] - 21s - loss: 0.0125 - acc: 0.9960 - val_loss: 0.0365 - val_acc: 0.9907
Epoch 5/8
60000/60000 [==============================] - 20s - loss: 0.0116 - acc: 0.9964 - val_loss: 0.0275 - val_acc: 0.9932
Epoch 6/8
60000/60000 [==============================] - 20s - loss: 0.0077 - acc: 0.9976 - val_loss: 0.0179 - val_acc: 0.9947
Epoch 7/8
60000/60000 [==============================] - 20s - loss: 0.0087 - acc: 0.9972 - val_loss: 0.0292 - val_acc: 0.9927
Epoch 8/8
60000/60000 [==============================] - 20s - loss: 0.0077 - acc: 0.9977 - val_loss: 0.0270 - 

# Reducing Overfitting

In [9]:
model = get_model()

In [15]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3, zoom_range=0.08, height_shift_range=0.08)
batches = gen.flow(x_train, y_train, batch_size=64)
val_batches = gen.flow(x_test, y_test, batch_size=64)

In [16]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 20s - loss: 0.1973 - acc: 0.9369 - val_loss: 0.0700 - val_acc: 0.9793


In [17]:
model.optimizer.lr = 0.1

In [18]:
model.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 19s - loss: 0.0696 - acc: 0.9783 - val_loss: 0.0535 - val_acc: 0.9837
Epoch 2/4
60000/60000 [==============================] - 19s - loss: 0.0560 - acc: 0.9819 - val_loss: 0.0442 - val_acc: 0.9853
Epoch 3/4
60000/60000 [==============================] - 19s - loss: 0.0466 - acc: 0.9857 - val_loss: 0.0447 - val_acc: 0.9839
Epoch 4/4
60000/60000 [==============================] - 19s - loss: 0.0438 - acc: 0.9870 - val_loss: 0.0410 - val_acc: 0.9871


In [19]:
model.optimizer.lr = 0.01

In [20]:
model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/8
60000/60000 [==============================] - 19s - loss: 0.0391 - acc: 0.9879 - val_loss: 0.0370 - val_acc: 0.9891
Epoch 2/8
60000/60000 [==============================] - 19s - loss: 0.0362 - acc: 0.9887 - val_loss: 0.0355 - val_acc: 0.9891
Epoch 3/8
60000/60000 [==============================] - 19s - loss: 0.0344 - acc: 0.9890 - val_loss: 0.0339 - val_acc: 0.9892
Epoch 4/8
60000/60000 [==============================] - 19s - loss: 0.0322 - acc: 0.9899 - val_loss: 0.0332 - val_acc: 0.9897
Epoch 5/8
60000/60000 [==============================] - 19s - loss: 0.0298 - acc: 0.9909 - val_loss: 0.0274 - val_acc: 0.9907
Epoch 6/8
60000/60000 [==============================] - 19s - loss: 0.0286 - acc: 0.9906 - val_loss: 0.0327 - val_acc: 0.9906
Epoch 7/8
60000/60000 [==============================] - 19s - loss: 0.0284 - acc: 0.9913 - val_loss: 0.0354 - val_acc: 0.9881
Epoch 8/8
60000/60000 [==============================] - 19s - loss: 0.0264 - acc: 0.9914 - val_loss: 0.0289 - 

In [21]:
model.optimizer.lr = 0.001

In [22]:
model.fit_generator(batches, batches.N, nb_epoch=14, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/14
60000/60000 [==============================] - 19s - loss: 0.0257 - acc: 0.9917 - val_loss: 0.0239 - val_acc: 0.9924
Epoch 2/14
60000/60000 [==============================] - 20s - loss: 0.0257 - acc: 0.9917 - val_loss: 0.0317 - val_acc: 0.9903
Epoch 3/14
60000/60000 [==============================] - 19s - loss: 0.0248 - acc: 0.9925 - val_loss: 0.0240 - val_acc: 0.9927
Epoch 4/14
60000/60000 [==============================] - 19s - loss: 0.0239 - acc: 0.9928 - val_loss: 0.0340 - val_acc: 0.9902
Epoch 5/14
60000/60000 [==============================] - 19s - loss: 0.0226 - acc: 0.9928 - val_loss: 0.0289 - val_acc: 0.9908
Epoch 6/14
60000/60000 [==============================] - 19s - loss: 0.0221 - acc: 0.9933 - val_loss: 0.0238 - val_acc: 0.9921
Epoch 7/14
60000/60000 [==============================] - 20s - loss: 0.0232 - acc: 0.9927 - val_loss: 0.0274 - val_acc: 0.9912
Epoch 8/14
60000/60000 [==============================] - 19s - loss: 0.0203 - acc: 0.9934 - val_loss: 0

In [23]:
model.optimizer.lr = 0.0001

In [24]:
model.fit_generator(batches, batches.N, nb_epoch=10, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/10
60000/60000 [==============================] - 19s - loss: 0.0182 - acc: 0.9944 - val_loss: 0.0274 - val_acc: 0.9930
Epoch 2/10
60000/60000 [==============================] - 19s - loss: 0.0210 - acc: 0.9934 - val_loss: 0.0284 - val_acc: 0.9921
Epoch 3/10
60000/60000 [==============================] - 19s - loss: 0.0190 - acc: 0.9940 - val_loss: 0.0276 - val_acc: 0.9914
Epoch 4/10
60000/60000 [==============================] - 19s - loss: 0.0181 - acc: 0.9943 - val_loss: 0.0325 - val_acc: 0.9905
Epoch 5/10
60000/60000 [==============================] - 19s - loss: 0.0182 - acc: 0.9943 - val_loss: 0.0269 - val_acc: 0.9922
Epoch 6/10
60000/60000 [==============================] - 19s - loss: 0.0190 - acc: 0.9940 - val_loss: 0.0263 - val_acc: 0.9921
Epoch 7/10
60000/60000 [==============================] - 19s - loss: 0.0175 - acc: 0.9946 - val_loss: 0.0240 - val_acc: 0.9931
Epoch 8/10
60000/60000 [==============================] - 19s - loss: 0.0165 - acc: 0.9947 - val_loss: 0

# Batchnorm + data Augmentation

In [25]:
def get_model_bn():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(), 
            BatchNormalization(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return  model

In [26]:
model = get_model_bn()

In [27]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 45s - loss: 0.1622 - acc: 0.9496 - val_loss: 0.0631 - val_acc: 0.9785


In [28]:
model.optimizer.lr = 0.1

In [29]:
model.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 45s - loss: 0.0712 - acc: 0.9780 - val_loss: 0.0554 - val_acc: 0.9815
Epoch 2/4
60000/60000 [==============================] - 45s - loss: 0.0592 - acc: 0.9813 - val_loss: 0.0460 - val_acc: 0.9841
Epoch 3/4
60000/60000 [==============================] - 45s - loss: 0.0529 - acc: 0.9837 - val_loss: 0.0362 - val_acc: 0.9883
Epoch 4/4
60000/60000 [==============================] - 45s - loss: 0.0470 - acc: 0.9850 - val_loss: 0.0339 - val_acc: 0.9897


In [30]:
model.optimizer.lr = 0.01

In [31]:
model.fit_generator(batches, batches.N, nb_epoch=12, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/12
60000/60000 [==============================] - 44s - loss: 0.0448 - acc: 0.9863 - val_loss: 0.0453 - val_acc: 0.9861
Epoch 2/12
60000/60000 [==============================] - 45s - loss: 0.0408 - acc: 0.9874 - val_loss: 0.0377 - val_acc: 0.9879
Epoch 3/12
60000/60000 [==============================] - 45s - loss: 0.0399 - acc: 0.9875 - val_loss: 0.0320 - val_acc: 0.9902
Epoch 4/12
60000/60000 [==============================] - 45s - loss: 0.0352 - acc: 0.9883 - val_loss: 0.0369 - val_acc: 0.9874
Epoch 5/12
60000/60000 [==============================] - 45s - loss: 0.0337 - acc: 0.9889 - val_loss: 0.0289 - val_acc: 0.9909
Epoch 6/12
60000/60000 [==============================] - 45s - loss: 0.0346 - acc: 0.9892 - val_loss: 0.0324 - val_acc: 0.9900
Epoch 7/12
60000/60000 [==============================] - 44s - loss: 0.0297 - acc: 0.9908 - val_loss: 0.0293 - val_acc: 0.9907
Epoch 8/12
60000/60000 [==============================] - 44s - loss: 0.0285 - acc: 0.9907 - val_loss: 0

In [32]:
model.optimizer.lr = 0.001

In [33]:
model.fit_generator(batches, batches.N, nb_epoch=12, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/12
60000/60000 [==============================] - 45s - loss: 0.0261 - acc: 0.9914 - val_loss: 0.0285 - val_acc: 0.9927
Epoch 2/12
60000/60000 [==============================] - 44s - loss: 0.0226 - acc: 0.9931 - val_loss: 0.0181 - val_acc: 0.9939
Epoch 3/12
60000/60000 [==============================] - 44s - loss: 0.0233 - acc: 0.9929 - val_loss: 0.0250 - val_acc: 0.9909
Epoch 4/12
60000/60000 [==============================] - 45s - loss: 0.0232 - acc: 0.9922 - val_loss: 0.0256 - val_acc: 0.9921
Epoch 5/12
60000/60000 [==============================] - 45s - loss: 0.0226 - acc: 0.9927 - val_loss: 0.0279 - val_acc: 0.9903
Epoch 6/12
60000/60000 [==============================] - 44s - loss: 0.0210 - acc: 0.9932 - val_loss: 0.0244 - val_acc: 0.9929
Epoch 7/12
60000/60000 [==============================] - 45s - loss: 0.0224 - acc: 0.9931 - val_loss: 0.0326 - val_acc: 0.9903
Epoch 8/12
60000/60000 [==============================] - 45s - loss: 0.0208 - acc: 0.9933 - val_loss: 0

# Batchnorm + dropout + dataaugmentation

In [34]:
def get_model_bn_do():
    model = Sequential([
            Lambda(normalise, input_shape=x_train.shape[1:]),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(), 
            BatchNormalization(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dropout(0.5),
            Dense(10, activation='softmax')
        ])
    
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return  model

In [35]:
model = get_model_bn_do()

In [36]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
60000/60000 [==============================] - 44s - loss: 0.2105 - acc: 0.9345 - val_loss: 0.0700 - val_acc: 0.9787


In [37]:
model.optimizer.lr = 0.1

In [38]:
model.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/4
60000/60000 [==============================] - 45s - loss: 0.0924 - acc: 0.9712 - val_loss: 0.0519 - val_acc: 0.9844
Epoch 2/4
60000/60000 [==============================] - 44s - loss: 0.0771 - acc: 0.9769 - val_loss: 0.0408 - val_acc: 0.9869
Epoch 3/4
60000/60000 [==============================] - 44s - loss: 0.0652 - acc: 0.9793 - val_loss: 0.0365 - val_acc: 0.9869
Epoch 4/4
60000/60000 [==============================] - 44s - loss: 0.0631 - acc: 0.9807 - val_loss: 0.0359 - val_acc: 0.9884


In [39]:
model.optimizer.lr = 0.01

In [40]:
model.fit_generator(batches, batches.N, nb_epoch=12, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/12
60000/60000 [==============================] - 44s - loss: 0.0595 - acc: 0.9821 - val_loss: 0.0393 - val_acc: 0.9878
Epoch 2/12
60000/60000 [==============================] - 44s - loss: 0.0529 - acc: 0.9833 - val_loss: 0.0344 - val_acc: 0.9891
Epoch 3/12
60000/60000 [==============================] - 45s - loss: 0.0515 - acc: 0.9841 - val_loss: 0.0356 - val_acc: 0.9888
Epoch 4/12
60000/60000 [==============================] - 44s - loss: 0.0500 - acc: 0.9847 - val_loss: 0.0348 - val_acc: 0.9885
Epoch 5/12
60000/60000 [==============================] - 44s - loss: 0.0459 - acc: 0.9861 - val_loss: 0.0310 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 44s - loss: 0.0450 - acc: 0.9866 - val_loss: 0.0323 - val_acc: 0.9883
Epoch 7/12
60000/60000 [==============================] - 44s - loss: 0.0403 - acc: 0.9876 - val_loss: 0.0254 - val_acc: 0.9921
Epoch 8/12
60000/60000 [==============================] - 44s - loss: 0.0397 - acc: 0.9881 - val_loss: 0

In [41]:
model.optimizer.lr = 0.001

In [42]:
model.fit_generator(batches, batches.N, nb_epoch=12, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/12
60000/60000 [==============================] - 45s - loss: 0.0342 - acc: 0.9895 - val_loss: 0.0240 - val_acc: 0.9916
Epoch 2/12
60000/60000 [==============================] - 44s - loss: 0.0348 - acc: 0.9896 - val_loss: 0.0330 - val_acc: 0.9900
Epoch 3/12
60000/60000 [==============================] - 44s - loss: 0.0317 - acc: 0.9900 - val_loss: 0.0265 - val_acc: 0.9916
Epoch 4/12
60000/60000 [==============================] - 44s - loss: 0.0290 - acc: 0.9907 - val_loss: 0.0274 - val_acc: 0.9908
Epoch 5/12
60000/60000 [==============================] - 44s - loss: 0.0325 - acc: 0.9901 - val_loss: 0.0240 - val_acc: 0.9928
Epoch 6/12
60000/60000 [==============================] - 45s - loss: 0.0310 - acc: 0.9903 - val_loss: 0.0267 - val_acc: 0.9920
Epoch 7/12
60000/60000 [==============================] - 44s - loss: 0.0318 - acc: 0.9905 - val_loss: 0.0231 - val_acc: 0.9930
Epoch 8/12
60000/60000 [==============================] - 44s - loss: 0.0288 - acc: 0.9911 - val_loss: 0

# Ensemble

In [43]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)
    model.optimizer.lr = 0.1
    model.fit_generator(batches, batches.N, nb_epoch=4, validation_data=val_batches, nb_val_samples=val_batches.N)
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.N, nb_epoch=12, validation_data=val_batches, nb_val_samples=val_batches.N)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.N, nb_epoch=18, validation_data=val_batches, nb_val_samples=val_batches.N)
    return model

In [45]:
models = [fit_model() for i in range(6)]

Epoch 1/1
60000/60000 [==============================] - 45s - loss: 0.2180 - acc: 0.9348 - val_loss: 0.0712 - val_acc: 0.9777
Epoch 1/4
60000/60000 [==============================] - 45s - loss: 0.0922 - acc: 0.9705 - val_loss: 0.0595 - val_acc: 0.9809
Epoch 2/4
60000/60000 [==============================] - 45s - loss: 0.0785 - acc: 0.9757 - val_loss: 0.0358 - val_acc: 0.9888
Epoch 3/4
60000/60000 [==============================] - 45s - loss: 0.0663 - acc: 0.9793 - val_loss: 0.0496 - val_acc: 0.9849
Epoch 4/4
60000/60000 [==============================] - 46s - loss: 0.0622 - acc: 0.9808 - val_loss: 0.0394 - val_acc: 0.9870
Epoch 1/12
60000/60000 [==============================] - 45s - loss: 0.0606 - acc: 0.9815 - val_loss: 0.0442 - val_acc: 0.9848
Epoch 2/12
60000/60000 [==============================] - 45s - loss: 0.0526 - acc: 0.9840 - val_loss: 0.0336 - val_acc: 0.9894
Epoch 3/12
60000/60000 [==============================] - 45s - loss: 0.0490 - acc: 0.9849 - val_loss: 0.0298

In [46]:
path = 'data/mnsit/'
model_path = path + 'model/'

In [47]:
for i, m in enumerate(models):
    m.save_weights(model_path+'mnsit_ensemble'+str(i)+'.pkl')

In [48]:
evals = [m.evaluate(x_test, y_test, batch_size=256) for m in models]

 9984/10000 [============================>.] - ETA: 0s

In [49]:
evals = np.array(evals)

In [50]:
evals.mean(axis=0)

array([ 0.0139,  0.9957])

In [51]:
all_preds = [m.predict(x_test, batch_size=256) for m in models]

In [52]:
all_preds = np.stack(all_preds)

In [53]:
avg_preds = all_preds.mean(axis=0)

In [54]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.996999979019165, dtype=float32)